# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
%pip install azureml-train-automl-runtime -q
%pip install azureml-sdk -q
%pip install azureml-widgets -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dask-sql 2024.5.0 requires dask[dataframe]>=2024.4.1, but you have dask 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires distributed>=2024.4.1, but you have distributed 2023.2.0 which is incompatible.
dask-sql 2024.5.0 requires pandas>=1.4.0, but you have pandas 1.3.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Set up Workspace and Experiment

In [2]:
from azureml.core import Workspace, Experiment, Dataset
ws = Workspace.from_config()
experiment_name = 'azmlproject3'

experiment = Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = experiment.start_logging()

Workspace name: quick-starts-ws-269873
Azure region: westus2
Subscription id: 3e42d11f-d64d-4173-af9b-12ecaa1030b3
Resource group: aml-quickstarts-269873


## Set up compute cluster

In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.exceptions import ComputeTargetException 

cluster_name = "azmlproject3"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target, use it:', cluster_name)
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D3_V2', max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

# get a detailed status for the current cluster using get_status()
print(compute_target.get_status().serialize())

Found existing compute target, use it: azmlproject3

Running
{'errors': [], 'creationTime': '2024-11-18T05:10:04.156291+00:00', 'createdBy': {'userObjectId': '85bb6c1f-e518-4125-92e4-883b18d39235', 'userTenantId': '660b3398-b80e-49d2-bc5b-ac1dc93b5254', 'userName': None}, 'modifiedTime': '2024-11-18T05:14:12.186558+00:00', 'state': 'Running', 'vmSize': 'Standard_D3_v2'}


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
from azureml.widgets import RunDetails
from azureml.train.hyperdrive import PrimaryMetricGoal, BanditPolicy, RandomParameterSampling, HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# choose random sampling
hyperparameter_sampling = RandomParameterSampling(
    {
        '--max_iter': choice(50, 75, 100),
        '--C': uniform(0.1, 10)
    }
)
# specify early stopping policy
early_termination_policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

src = ScriptRunConfig(source_directory='.', script='train.py', compute_target=compute_target, environment=sklearn_env)

hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=hyperparameter_sampling,
                                     policy=early_termination_policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=5)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [5]:
#TODO: Submit your experiment
run = experiment.submit(hyperdrive_config)

In [6]:
# Show the RunDetails widget
RunDetails(run).show()
run.wait_for_completion(show_output=True)

2024-11-18 05:31:19.146123: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-18 05:31:21.675309: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-18 05:31:22.531864: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-18 05:31:27.894978: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-18 05:31:40.840389: W tensorflow/comp

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff
Web View: https://ml.azure.com/runs/HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff?wsid=/subscriptions/3e42d11f-d64d-4173-af9b-12ecaa1030b3/resourcegroups/aml-quickstarts-269873/workspaces/quick-starts-ws-269873&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-11-18T05:29:52.1814802Z][GENERATOR][DEBUG]Sampled 5 jobs from search space 
[2024-11-18T05:29:52.8827756Z][SCHEDULER][INFO]Scheduling job, id='HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_2' 
[2024-11-18T05:29:52.7811642Z][SCHEDULER][INFO]Scheduling job, id='HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_0' 
[2024-11-18T05:29:52.8817404Z][SCHEDULER][INFO]Scheduling job, id='HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_3' 
[2024-11-18T05:29:52.8837454Z][SCHEDULER][INFO]Scheduling job, id='HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_1' 
[2024-11-18T05:29:52.9294097Z][SCHEDULER][INFO]Scheduling job, id='HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_4' 
[2024-11-18T05:29:53.6728587Z]

{'runId': 'HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff',
 'target': 'azmlproject3',
 'status': 'Completed',
 'startTimeUtc': '2024-11-18T05:29:50.625479Z',
 'endTimeUtc': '2024-11-18T05:46:59.614018Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '1d94ac76-6299-40fc-8965-94cf3a204990',
  'user_agent': 'python/3.10.11 (Linux-5.15.0-1073-azure-x86_64-with-glibc2.31) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'best_child_run_id': 'HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_18',
  'score': '0.7327800829875518',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_18'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sd

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [7]:
best = run.get_best_run_by_primary_metric()
best_run_metrics = best.get_metrics()
print("Best configs: ",best_run_metrics)

Best configs:  {'Regularization:': 0.7638999380214355, 'Max iters:': 100, 'Accuracy': 0.7327800829875518}


In [8]:
best

Experiment,Id,Type,Status,Details Page,Docs Page
azmlproject3,HD_c4cda66d-c195-4aa9-a682-cfff1f1f44ff_18,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [9]:
#TODO: Save the best model
import joblib
# Get your best run and save the model from that run.
best_model = [x for x in best.get_file_names() if '.joblib' in x][0]
best.download_file(name=best_model, output_file_path='./training')
print('Model saved')

Model saved


In [10]:
# Find the best model file
#best_model = [x for x in best.get_file_names() if '.joblib' in x][0]
#best.download_file(name=best_model, output_file_path='./training/' + best_model)

# Load the saved model from the specified path
#model_path = './training/' + best_model
#model = joblib.load(model_path)

In [14]:
# Define the model filename (assuming best_model contains the full filename)
model_filename = os.path.basename(best_model)  # Extract just the filename

# Register the model with only the filename as the path
registered_model = best.register_model(
    model_path= 'outputs/model_accuracy_0.7327800829875518_C_0.7638999380214355_maxIter_100.joblib',  # Only specify the filename, as it’s within 'outputs/'
    model_name='best_hyperparameter',
    properties={'accuracy': best.get_metrics().get('accuracy')}
)

print('Model registered successfully!')

Model registered successfully!


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

